# Flight Radar API
- Non-official flight radar API for studies
- https://jeanextreme002.github.io/FlightRadarAPI/python/

### Import libraries

In [2]:
from pyspark.sql import functions as F
import sys, os
path = os.path.dirname(os.getcwd()) + '/BaseUtils/'
sys.path.append(os.path.abspath(path))

from hdfs_io import *

hdfs_obj = HDFS_IO()

In [3]:
# Instanciate FLightRadarAPI

from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

### Sao Paulo airport

In [4]:
gru_airport = fr_api.get_airport(code = "GRU", details = True)
print(type(gru_airport))
print(gru_airport)

# gru_airport_bounds = 

<class 'FlightRadar24.entities.airport.Airport'>
<(SBGR) Sao Paulo Guarulhos International Airport - Altitude: 2459 - Latitude: -23.431999 - Longitude: -46.469501>


### GRU Flights

In [6]:
gru_bounds = fr_api.get_bounds_by_point(-23.431999, -46.469501, 20000)
print(gru_bounds)

-23.252012783009913,-23.611740501591253,-46.66579902001932,-46.27373639848763


In [7]:
# bounds = latitude, longitude, radius
gru_bounds = fr_api.get_bounds_by_point(-23.431999, -46.469501, 20000)
gru_flights = fr_api.get_flights(bounds = gru_bounds)

print(type(gru_flights))
print(type(gru_flights[0]))
print(gru_flights[0])

<class 'list'>
<class 'FlightRadar24.entities.flight.Flight'>
<(T210) PP-FSR - Altitude: 2375 - Ground Speed: 9 - Heading: 192>


In [8]:
print(dir(gru_flights))

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [9]:
print(dir(gru_flights[0]))

['_Flight__get_info', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_default_text', 'aircraft_code', 'airline_iata', 'airline_icao', 'altitude', 'callsign', 'check_info', 'destination_airport_iata', 'get_altitude', 'get_distance_from', 'get_flight_level', 'get_ground_speed', 'get_heading', 'get_vertical_speed', 'ground_speed', 'heading', 'icao_24bit', 'id', 'latitude', 'longitude', 'number', 'on_ground', 'origin_airport_iata', 'registration', 'set_flight_details', 'squawk', 'time', 'vertical_speed']


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LocalSpark") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 3.5.0


In [6]:
zones_keys = list(zones.keys())
print(zones_keys)

# Convert the list of strings to a list of tuples
zones_tuples = [(zone,) for zone in zones_keys]

['europe', 'northamerica', 'southamerica', 'oceania', 'asia', 'africa', 'atlantic', 'maldives', 'northatlantic']


In [7]:
zones_df = spark.createDataFrame(zones_tuples, ["zone_name"])

In [8]:
zones_df.show()

+-------------+
|    zone_name|
+-------------+
|       europe|
| northamerica|
| southamerica|
|      oceania|
|         asia|
|       africa|
|     atlantic|
|     maldives|
|northatlantic|
+-------------+



In [9]:
destination_path = hdfs_obj.base_url + hdfs_obj.user_path + 'FlightRadarApi/zones'
print(destination_path)

hdfs://namenode:9000/user/jovyan/FlightRadarApi/zones


In [10]:
# zones_df.write.mode("overwrite").parquet(destination_path)
zones_df.write.parquet(destination_path, mode='overwrite')

Py4JJavaError: An error occurred while calling o45.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 8) (172.26.0.3 executor 0): java.net.ConnectException: Call From 032847bcde23/172.26.0.3 to namenode:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at org.apache.hadoop.net.N

In [ ]:
# Read from parquet zones

df_read_zones = spark.read.parquet(destination_path)
df_read_zones.show()

In [ ]:
# hdfs_obj.write_to_namenode(hdfs_obj, base_folder + '/zones')